In [ ]:
import pyvisa
import serial
import time

# Connect to the scope

# Create a Resource Manager
rm = pyvisa.ResourceManager()

# List all available resources
resources = rm.list_resources()

# Print addresses of all connected devices
if resources:
    print("Connected VISA resources:")
    for resource in resources:
        print(resource)
else:
    print("No VISA resources found.")

# Open the first resource and refer to it as 'scope'
if resources:
    scope = rm.open_resource(resources[0])  # Open the first resource
    print(f"\nDetails of the first device ({resources[0]}):")
    print(scope.query("*IDN?"))  # Send an identification query (optional)

# Initial setup of the scope

# Initial
channels = ['CHANnel1', 'CHANnel2', 'CHANnel3', 'CHANnel4']
for channel in channels:
    scope.write(f':{channel}:DISPlay ON')  # Activate channels
    scope.write(f':{channel}:SCALe 2')    # Set scale
    scope.write(f':{channel}:OFFSet -6')  # Set offset


In [ ]:
# Connect to the arduino
output_string = ""
for ini in range(40):
    output_string += f"{ini},{0};"
start = output_string +'\n'
ser = serial.Serial('COM4', 9600) 
time.sleep(2)
ser.write(start.encode())
time.sleep(1)

In [ ]:
# Here one can choose a specific value for each heater

# Serial port configuration
SERIAL_PORT = 'COM4'  # Replace with your serial port
BAUD_RATE = 9600

# Desired heater values grouped into layers, all set to 0
heater_values = {
    "Layer1": {
        0: 2.0,
        1: 2.0,
        2: 2.0,
        3: 2.0,
        4: 2.0,
        5: 2.0,
        6: 2.0
    },
    "Layer2": {
        7: 0.0,
        8: 0.0,
        9: 0.0,
        10: 0.0,
        11: 0.0,
        12: 0.0,
        13: 0.0
    },
    "Layer3": {
        14: 0.0,
        15: 0.0,
        16: 0.0,
        17: 0.0,
        18: 0.0,
        19: 0.0,
        20: 0.0
    },
    "Layer4": {
        21: 0.0,
        22: 0.0,
        23: 0.0,
        24: 0.0,
        25: 0.0,
        26: 0.0,
        27: 0.0
    },
    "Layer5": {
        28: 0.0,
        29: 0.0,
        30: 0.0,
        31: 0.0,
        32: 0.0,
        33: 0.0,
        34: 0.0
    }
}

try:
    # Open serial connection
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE)
    time.sleep(2)  # Allow time for the connection to stabilize

    for layer_name, layer_values in heater_values.items():
        voltage_message = "".join(f"{heater},{value};" for heater, value in layer_values.items()) + '\n'
        
        # Send the voltage message for the current layer
        print(f"Sending heater values for {layer_name}...")
        ser.write(voltage_message.encode())
        time.sleep(1)  # Allow time for the board to process

        # print(f"{layer_name} values sent successfully:")
        # for heater, value in layer_values.items():
        #     print(f"Heater {heater}: {value}V")

except serial.SerialException as e:
    print(f"Error: {e}")

finally:
    # Close the serial connection
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial connection closed.")


In [ ]:
# Read some values

# For measuring, I have the documentation in documents
output1 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel1')),5)
output2 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel2')),5)
output3 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel3')),5)
output4 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel4')),5)

print(output1)
print(output2)
print(output3)
print(output4)


In [ ]:
# Here I change the value of each heater to see how it affects, I save it to a csv

import serial
import time
import pyvisa
# Serial port configuration
SERIAL_PORT = 'COM4'  # Replace with your serial port
BAUD_RATE = 9600
# Initialize oscilloscope
rm = pyvisa.ResourceManager()
resources = rm.list_resources()
if not resources:
    raise Exception("No VISA resources found. Make sure the oscilloscope is connected.")
scope = rm.open_resource(resources[0])
scope.timeout = 5000  # Set timeout to 5 seconds
# Heater values
heater_values = {i: 0.0 for i in range(35)}  # 35 heaters in total
THRESHOLD = 1.0  # Define a threshold for significant change in outputs
# Function to send heater values
def send_heater_values(ser):
    voltage_message = "".join(f"{heater},{value};" for heater, value in heater_values.items()) + '\n'
    ser.write(voltage_message.encode())
    ser.flush()
    ser.reset_input_buffer()
    ser.reset_output_buffer()
    time.sleep(0.01)
# Function to measure outputs
def measure_outputs():
    try:
        output1 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel1')), 5)
        output2 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel2')), 5)
        output3 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel3')), 5)
        output4 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel4')), 5)
      
        return output1, output2, output3, output4
    except Exception as e:
        print(f"Error measuring outputs: {e}")
        return [None, None, None, None]
# Main loop
def main():
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE)
    time.sleep(2)
    voltage_range = [round(v * 0.5, 1) for v in range(0, 11)]  # 0.0, 0.5, ..., 5.0
    results = []  # Store all results in memory
    prev_outputs = [0.0, 0.0, 0.0, 0.0]  # Initialize previous outputs
    for heater in heater_values.keys():
        print(f"Testing Heater {heater}...")
        for key in heater_values.keys():
            heater_values [key] = 0.0
        for voltage in voltage_range:
            print(f"Sending voltage to Heater {heater}, Voltage {voltage}V...")
            heater_values[heater] = voltage
            send_heater_values(ser)
            print("Voltage sent. Measuring outputs...")
            outputs = measure_outputs()
            if outputs == [None, None, None, None]:
                print("Measurement failed. Skipping...")
                break
            # Compare with previous outputs for significant changes
            if any(abs(outputs[i] - prev_outputs[i]) > THRESHOLD for i in range(4)):
                print("Significant change detected. Adding delay...")
                time.sleep(1)  # Delay for stabilization
            # Save results in memory
            results.append([heater, voltage] + list(outputs))
            print(f"Heater {heater}, Voltage {voltage}V -> Outputs: {outputs}")
            prev_outputs = outputs  # Update previous outputs
        heater_values[heater] = 0.0
    ser.close()
    print("All cases tested. Saving results to CSV...")
    # Save all results to CSV at the end
    with open('individual_heater_test_log.csv', 'w', newline='') as csvfile:
        import csv
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Heater", "Voltage", "Output1", "Output2", "Output3", "Output4"])
        csvwriter.writerows(results)
    print("Results saved to 'individual_heater_test_log.csv'.")
if __name__ == "__main__":
    main()

In [ ]:
# So here I try 5,000 possible combinations of values, the problems is I was not controlling the first layers as inputs, but as another layer thinking I was gonna use 2 different lasers.
import serial
import time
import pyvisa
import itertools
import random
# Serial port configuration
SERIAL_PORT = 'COM4'  # Replace with your serial port
BAUD_RATE = 9600
# Initialize oscilloscope
rm = pyvisa.ResourceManager()
resources = rm.list_resources()
if not resources:
    raise Exception("No VISA resources found. Make sure the oscilloscope is connected.")
scope = rm.open_resource(resources[0])
scope.timeout = 5000  # Set timeout to 5 seconds
# Heater values grouped by layers
heater_values = {i: 0.0 for i in range(35)}  # 35 heaters in total
layers = {
    0: list(range(0, 7)),  # Layer 1: Heaters 0-6
    1: list(range(7, 14)),  # Layer 2: Heaters 7-13
    2: list(range(14, 21)),  # Layer 3: Heaters 14-20
    3: list(range(21, 28)),  # Layer 4: Heaters 21-27
    4: list(range(28, 35)),  # Layer 5: Heaters 28-34
}
# Reduced voltage range
voltage_range = [0.0, 2.5, 5.0]  # Low, mid, high voltages
# Define the number of random combinations to sample per layer
num_combinations_per_layer = 1000
# Function to send heater values
def send_heater_values(ser):
    voltage_message = "".join(f"{heater},{value};" for heater, value in heater_values.items()) + '\n'
    ser.write(voltage_message.encode())
    ser.flush()
    ser.reset_input_buffer()
    ser.reset_output_buffer()
    time.sleep(0.01)
# Function to measure outputs
def measure_outputs():
    try:
        output1 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel1')), 5)
        output2 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel2')), 5)
        output3 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel3')), 5)
        output4 = round(float(scope.query(':MEASure:STATistic:ITEM? CURRent,VMAX,CHANnel4')), 5)
        return output1, output2, output3, output4
    except Exception as e:
        print(f"Error measuring outputs: {e}")
        return [None, None, None, None]
# Main loop
def main():
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE)
    time.sleep(2)
    results = []  # Store all results in memory
    prev_outputs = [0.0, 0.0, 0.0, 0.0]  # Initialize previous outputs
    # Test a subset of layers (e.g., start with first two layers)
    for layer, heaters in layers.items():
        print(f"Testing Layer {layer}...")
        # Generate all combinations for the current layer and sample a subset
        all_combinations = list(itertools.product(voltage_range, repeat=len(heaters)))
        sampled_combinations = random.sample(all_combinations, min(num_combinations_per_layer, len(all_combinations)))
        for combination in sampled_combinations:
            # Reset all heaters to baseline
            for key in heater_values.keys():
                heater_values[key] = 0.0
            # Apply the combination to the current layer
            for heater, voltage in zip(heaters, combination):
                heater_values[heater] = voltage
            print(f"Testing combination {combination} for Layer {layer}...")
            send_heater_values(ser)
            print("Voltage sent. Measuring outputs...")
            outputs = measure_outputs()
            if outputs == [None, None, None, None]:
                print("Measurement failed. Skipping...")
                break
            # Save results in memory
            results.append([layer, combination] + list(outputs))
            print(f"Layer {layer}, Combination {combination} -> Outputs: {outputs}")
    ser.close()
    print("All cases tested. Saving results to CSV...")
    # Save all results to CSV at the end
    with open('layer_test_log.csv', 'w', newline='') as csvfile:
        import csv
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Layer", "Combination", "Output1", "Output2", "Output3", "Output4"])
        csvwriter.writerows(results)
    print("Results saved to 'layer_test_log.csv'.")
if __name__ == "__main__":
    main()

